<h3> Regression </h3>

This notebook demonstrates the regression task in data analytics.

In all examples, we will use the `students_reg` dataset. The data was collected from students at week 12 of their 2nd freshman semester. The target is to predict their final first year GPA based on their other information.

We will first load the data in and process using pipeline like in the previous module.

In [1]:
%spark2.pyspark

#path to data
hdfs_path = '/tmp/data/'
data_file = 'students_reg.csv'
split_ratio = [0.7, 0.3]
drop_cols = ['StudentID', 'FirstName', 'LastName']
integer_cols = ['FamilyIncome', 'TotalAbsence']
string_cols = ['Major', 'State']
numeric_cols = ['HighSchoolGPA','FamilyIncome','AvgDailyStudyTime','TotalAbsence']
target = 'FirstYearGPA'

from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

#read data
data = spark.read.options(header='True',inferSchema='True',delimiter=',').csv(hdfs_path+data_file)

#drop columns
data = data.drop(*drop_cols)

#cast integer columns to double
for c in integer_cols:
    data = data.withColumn(c, col(c).cast(DoubleType()))
    
#train-test split
data_train, data_test = data.randomSplit(split_ratio)

from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

###one hot encode the categorical columns
encoders = []
for c in string_cols:
    encoders.append(StringIndexer(inputCol=c, outputCol=c+'Index', handleInvalid='keep'))
    encoders.append(OneHotEncoder(inputCol=c+'Index', outputCol=c+'Codes'))

###impute the numeric columns
imputer = Imputer(inputCols = numeric_cols, outputCols = [c+'Imp' for c in numeric_cols], strategy = 'median')

###standardization
num_assembler = VectorAssembler(inputCols=[c+'Imp' for c in numeric_cols], outputCol='imputed')
scaler = StandardScaler(inputCol = 'imputed', outputCol = 'scaled')

###combine results
assembler = VectorAssembler(inputCols=[c+'Codes' for c in string_cols]+['scaled'], outputCol='features')



###build pipeline
pipeline = Pipeline(stages = encoders + [imputer, num_assembler, scaler, assembler])

###train pipeline
pipeline_trained = pipeline.fit(data_train)

###process training data annd testing data
train_prc = pipeline_trained.transform(data_train).select(target,'features')
test_prc = pipeline_trained.transform(data_test).select(target,'features')

## Modeling

We will tune and test some common regression models available in PySpark:
- Linear regression
- Decision tree
- Random forest
- Gradient boosting model

We can automate the search for the best hyperparamters with Cross Validation Grid Search. In pyspark, we use a combination of `ParamGridBuilder` and `CrossValidator`. The steps are as follows
1. Create an empty model
2. Create the parameter grid with `ParamGridBuilder`. Each hyperparameter requires a different `addGrid()` call; multiple `addGrid()` can be chained.
3. Create the `CrossValidator` object
    - `estimator`: the empty model
    - `estimatorParamMaps`: the parameter grid
    - `evaluator`: the evaluator object (`RegressionEvaluator` for regression)
    - `numFolds`: number of folds for cross validation
4. Train the CrossValidator with fit()

First, we import general libaries and create an evaluator. `metricName` are commonly `mse` or `r2`. Lower MSE and higher R2 mean better models.

In [3]:
%spark2.pyspark

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

eval_mse = RegressionEvaluator(labelCol=target, metricName='mse')
eval_r2 = RegressionEvaluator(labelCol=target, metricName='r2')

### Linear regression

Linear regression has two hyperparameters, `regParam` and `elasticNetParam`, to tune

In [5]:
%spark2.pyspark

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol=target)

paramGridLR = ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.1, 1.0, 10.0])\
                                .addGrid(lr.elasticNetParam, [0.25, 0.5, 0.75])\
                                .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGridLR,
                          evaluator=eval_r2,
                          numFolds=10) 

cvLR = crossval.fit(train_prc)

train_pred_cvLR = cvLR.transform(train_prc)
test_pred_cvLR = cvLR.transform(test_prc)

print('cross-validation linear regression')
print('training MSE: ', eval_mse.evaluate(train_pred_cvLR))
print('training R2: ', eval_r2.evaluate(train_pred_cvLR))
print('testing MSE: ', eval_mse.evaluate(test_pred_cvLR))
print('testing R2: ', eval_r2.evaluate(test_pred_cvLR))

cross-validation linear regression
('training MSE: ', 0.04674396580841727)
('training R2: ', 0.8446809698340348)
('testing MSE: ', 0.04695639677048426)
('testing R2: ', 0.8484984403489446)


### Decision tree

The two important hyperparameters to tune for decision tree are `maxDepth` and `minInstancesPerNode`

In [7]:
%spark2.pyspark

from pyspark.ml.regression import DecisionTreeRegressor

#create empty model
dt = DecisionTreeRegressor(featuresCol='features', labelCol=target)

#parameter grid for decision tree
paramGridTree = ParamGridBuilder().addGrid(dt.maxDepth, [3, 5, 7])\
                                  .addGrid(dt.minInstancesPerNode, [10, 20, 30]).build()

#cross validator
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGridTree,
                          evaluator=eval_r2,
                          numFolds=3) 

#perform the search
cvTree = crossval.fit(train_prc)

#test the tuned model
train_pred_cvTree = cvTree.transform(train_prc)
test_pred_cvTree = cvTree.transform(test_prc)

print('cross-validation decision tree')
print('training MSE: ', eval_mse.evaluate(train_pred_cvTree))
print('training R2: ', eval_r2.evaluate(train_pred_cvTree))
print('testing MSE: ', eval_mse.evaluate(test_pred_cvTree))
print('testing R2: ', eval_r2.evaluate(test_pred_cvTree))

cross-validation decision tree
('training MSE: ', 0.04290978170608536)
('training R2: ', 0.8574210475307527)
('testing MSE: ', 0.07325860074639325)
('testing R2: ', 0.7636362022158183)


### Random Forest

Random Forest is an ensemble of decision trees and usually yields better performances. 

Similar to a tree, we need to tune maxDepth and `minInstancesPerNode`. We also need to tune `numTrees` - the number of trees in a forest model.

In [9]:
%spark2.pyspark

from pyspark.ml.regression import RandomForestRegressor

#initialize model
rf = RandomForestRegressor(featuresCol='features', labelCol=target)

#paramter grid
paramGridForest = ParamGridBuilder().addGrid(rf.numTrees, [10, 30, 50])\
                                    .addGrid(dt.maxDepth, [3, 5, 7])\
                                    .addGrid(dt.minInstancesPerNode, [10, 20, 30])\
                                    .build()
#cross validator
crossval = CrossValidator(estimator = rf,
                          estimatorParamMaps = paramGridForest,
                          evaluator = eval_r2,
                          numFolds = 3) 

#perform tuning
cvForest = crossval.fit(train_prc)

#test the tuned model
train_pred_cvForest = cvForest.transform(train_prc)
test_pred_cvForest = cvForest.transform(test_prc)

print('cross-validation random forest')
print('training MSE: ', eval_mse.evaluate(train_pred_cvForest))
print('training R2: ', eval_r2.evaluate(train_pred_cvForest))
print('testing MSE: ', eval_mse.evaluate(test_pred_cvForest))
print('testing R2: ', eval_r2.evaluate(test_pred_cvForest))

cross-validation random forest
('training MSE: ', 0.06105781817749406)
('training R2: ', 0.7971194583222437)
('testing MSE: ', 0.08102633499630833)
('testing R2: ', 0.7385741460369937)


<h4>Gradient Boosting Model</h4>

Gradient boosting model (GBT) is similar to random forest, however, each tree is added to the ensemble to minimize the current training error instead of randomly.

GBT models still have `maxDepth` and minInstancesPerNode to tune, however, we do not tune the `numTrees` anymore.

In [11]:
%spark2.pyspark

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


gbt = GBTRegressor(featuresCol='features', labelCol=target)

paramGridGBT = ParamGridBuilder().addGrid(gbt.maxDepth, [3, 5, 7])\
                                 .addGrid(gbt.minInstancesPerNode, [10, 20, 30])\
                                 .build()

crossval = CrossValidator(estimator = gbt,
                          estimatorParamMaps = paramGridGBT,
                          evaluator = eval_r2,
                          numFolds = 3) 

cvGBT = crossval.fit(train_prc)

train_pred_cvGBT = cvGBT.transform(train_prc)
test_pred_cvGBT = cvGBT.transform(test_prc)

print('cross-validation random forest')
print('training MSE: ', eval_mse.evaluate(train_pred_cvGBT))
print('training R2: ', eval_r2.evaluate(train_pred_cvGBT))
print('testing MSE: ', eval_mse.evaluate(test_pred_cvGBT))
print('testing R2: ', eval_r2.evaluate(test_pred_cvGBT))

cross-validation random forest
('training MSE: ', 0.044394343701222395)
('training R2: ', 0.8524882027171358)
('testing MSE: ', 0.06115345735534525)
('testing R2: ', 0.802692608364431)
